In [1]:
from google.colab import drive, files
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [1]:
import os
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xgboost as xgb
import ast 
import sklearn.metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

In [2]:
root_path = os.path.join(os.getcwd(), "drive", "My Drive/orie4741 final") # replace based on your Google drive organization
dataset_path = os.path.join(root_path, "data/") # same here

# We create a dataframe of our cleaned data

In [690]:
data = pd.read_csv('cleaneddatav3.csv')
data

,Unnamed: 0,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,...,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,is_success
0,6054,0.97200,['David Bauer'],0.567,313293,0.227,0,0w0D8H1ubRerCXHWYJkinO,0.601000,10,...,-13.441,1,Shout to the Lord,47,2000,0.0290,136.123,0.0396,2000,0
1,6055,0.32100,['Etta James'],0.821,360240,0.418,0,4JVeqfE2tpi7Pv63LJZtPh,0.000372,9,...,-9.841,0,Miss You,51,2000-12-12,0.0407,117.382,0.8030,2000,0
2,6056,0.00659,['Quasimoto'],0.706,202507,0.602,1,5pxtdhLAi0RTh1gNqhGMNA,0.000138,11,...,-8.306,0,Real Eyes,44,2000-06-13,0.3420,89.692,0.4790,2000,0
3,6057,0.00390,['Millencolin'],0.368,173360,0.977,0,3jRsoe4Vkxa4BMYqGHX8L0,0.000000,11,...,-2.757,0,Penguins & Polarbears,52,2000-02-22,0.1270,165.889,0.5480,2000,0
4,6058,0.12200,['Steve Chou'],0.501,344200,0.511,0,4mronxcllhfyhBRqyZi8kU,0.000000,7,...,-9.836,0,黃昏,53,2000-12-25,0.0291,78.045,0.1130,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,168804,0.00303,['Hannah Montana'],0.720,155600,0.913,0,5J0d605KKCz4x8zbUKTaPe,0.000062,7,...,-3.670,1,Let's Get Crazy,52,2009-01-01,0.0491,124.997,0.8000,2009,1
19996,168805,0.01310,['Eminem'],0.635,304907,0.712,1,1PokNDiTBQZgXci6rn2BVE,0.000000,4,...,-5.151,0,Music Box,52,2009-05-15,0.1100,78.598,0.4580,2009,1
19997,168806,0.08730,['Tribal Seeds'],0.746,312200,0.695,0,2mVnrvWkiFTERSE2SL4QZ7,0.000073,1,...,-5.115,0,The Harvest,47,2009-05-26,0.0817,164.022,0.5910,2009,0
19998,168807,0.01530,['Wisin & Yandel'],0.711,259067,0.860,0,27fNEGLckXrSf9ypb7d0BM,0.000000,8,...,-3.507,1,Te Siento,52,2009-01-01,0.0414,109.946,0.4040,2009,0


# Then, we download the word embeddings (this may take a while).

In [4]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

--2020-12-10 22:18:16--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-12-10 22:18:16--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2020-12-10 22:18:16--  http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [appli

In [ ]:
!unzip glove.twitter.27B.zip

Archive:  glove.twitter.27B.zip
replace glove.twitter.27B.25d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
!ls

(MIT-Pappalardo Series in Mechanical Engineering) Gang Chen - Nanoscale Energy Transport and Conversion_ A Parallel Treatment of Electrons, Molecules, Phonons, and Photons-Oxford University Press (200 (1).djvu
(MIT-Pappalardo Series in Mechanical Engineering) Gang Chen - Nanoscale Energy Transport and Conversion_ A Parallel Treatment of Electrons, Molecules, Phonons, and Photons-Oxford University Press (200 (2).djvu
(MIT-Pappalardo Series in Mechanical Engineering) Gang Chen - Nanoscale Energy Transport and Conversion_ A Parallel Treatment of Electrons, Molecules, Phonons, and Photons-Oxford University Press (200.djvu
01 Intro and Solar lecture.pptx
040605.pdf
1-s2.0-S0379677918304077-main.pdf
1-s2.0-S0927024819306130-main.pdf
1-s2.0-S0927025619301922-main.pdf
1-s2.0-S2211285520309575-main.pdf
1-s2.0-S2211285520311204-main (1).pdf
1-s2.0-S2211285520311204-main.pdf
1-s2.0-S2352847817300515-main.pdf
1-s2.0-S2542435119302570-main (1).pdf
1-s2.0-S2542435119302570-main.pdf
1807.08012.pdf
18

In [777]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.twitter.27B.200d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 1193514 word vectors.


# We then write functions to find and sum the word embeddings of artist names and song names. To include song names, uncomment the commented lines below.

In [798]:
def convertToEm(artists):
  arr = ast.literal_eval(artists)
  embeds = np.zeros(200)
  for i in arr:
    strs = i.split()
    for w in strs:
      check = w.lower()
      if check in embeddings_index:
        embeds = np.add(embeds, embeddings_index[check])
  return embeds


In [ ]:
words = set()
hasEmbeds = set()
for index, r in data.iterrows():
  arr = ast.literal_eval(r.artists)
  for i in arr:
    strs = i.split()
    for w in strs:
      check = w.lower()
      if check in embeddings_index:
        hasEmbeds.add(check)
  

In [ ]:
df_names = []
for i in data['name']:
  df_names.append(i.replace("'","").replace('(','').replace(')','').replace(" -",'').replace(',','').replace("[",'').replace("]",'').lower().split())

In [ ]:
words = set()
hasEmbeds2 = set()
for r in df_names:
  for k in r:
    check = k.lower()
    if check in embeddings_index:
      hasEmbeds2.add(check)

In [ ]:
def convertToEm2(names):
  embeds = np.zeros(25)
  arr = names
  for i in arr:
    strs = i.split()
    for w in strs:
      check = w.lower()
      if check in embeddings_index:
        embeds = np.add(embeds, embeddings_index[check])
  return embeds


In [ ]:
data['vector'] = data['artists'].apply(convertToEm)
data['vector']

In [853]:
#data['vector2'] = df_names
#data['vector2'] = data['vector2'].apply(convertToEm2)

In [ ]:
data = data.drop(columns = ['Unnamed: 0', 'artists', 'id', 'name', 'release_date','popularity'])

In [ ]:
df = pd.DataFrame(data['vector'].values.tolist())
#df2 = pd.DataFrame(data['vector2'].values.tolist())

In [855]:
#df2.rename(columns=dict(zip(df2.columns, df2.columns+200)), inplace=True)

In [856]:
data = pd.concat([data, df], axis=1)
#data = pd.concat([data, df2], axis=1)
data

In [ ]:
data = data.drop('vector', axis=1)
#data = data.drop('vector2', axis=1)

In [ ]:
X = data.drop('is_success',axis = 1)
y = data['is_success']

# Next we split the data into train and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=90

# Next we upsample our training set. To downsample, replace df_min with df_maj and vice versa in all the lines below the commented line.

In [861]:
from sklearn.utils import resample
import random
X_train['is_success'] = y_train
df_train = X_train
df_maj = df_train[df_train['is_success']==0]
df_min = df_train[df_train['is_success']==1]
#############################################################################################
n = len(df_maj)
df_sampled = resample(df_min,replace=True, n_samples=n,random_state=123)
df_comb = df_maj.append(df_sampled).sample(frac=1)
X_train = df_comb.drop('is_success',axis = 1)
y_train = df_comb['is_success']

<ipython-input-861-1068ffabfdff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['is_success'] = y_train


In [870]:
X_train

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,...,190,191,192,193,194,195,196,197,198,199
21682,0.1690,0.480,206587,0.928,0,0.000000,4,0.0479,-5.501,1,...,0.951237,0.682730,0.877220,-0.927200,0.676982,-0.539576,0.564151,0.80333,-0.601177,-1.071780
29867,0.6980,0.548,237333,0.402,0,0.000114,2,0.1350,-8.727,1,...,-0.082449,0.267909,0.798040,-0.148500,-0.171560,-0.242690,1.766840,0.99689,-0.871880,-0.766085
10320,0.0270,0.879,156284,0.732,0,0.000000,11,0.0656,-5.016,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
21725,0.2440,0.798,293053,0.443,1,0.000000,2,0.3260,-6.062,1,...,0.715740,0.407140,-0.808940,0.295330,0.261620,-0.567750,0.063045,-0.42014,0.082703,-0.607620
5683,0.6650,0.357,188133,0.361,0,0.005920,0,0.0682,-12.140,1,...,-0.780900,0.562960,0.706170,-0.405420,-0.151110,0.038366,0.193380,-0.14097,-0.636540,-0.405560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12984,0.0966,0.730,239627,0.700,0,0.000144,9,0.1140,-8.648,1,...,-0.219859,-0.581400,-0.131920,-0.160220,-0.460840,-0.289830,-0.089420,-0.12725,-0.509018,-0.031100
15153,0.1900,0.407,295320,0.678,0,0.000000,2,0.3420,-6.135,1,...,0.049430,-0.647250,0.291100,0.251444,1.770500,0.037858,2.132870,0.30359,-0.641000,-1.240430
243,0.3720,0.680,244507,0.590,0,0.000027,9,0.1250,-7.601,0,...,-0.197020,0.438283,0.580803,1.329000,0.002350,-1.195640,0.762230,0.17651,-0.752000,-1.202950
22594,0.2050,0.870,251989,0.464,1,0.000000,4,0.0902,-8.337,1,...,-0.649394,1.501188,-1.390750,0.919300,-0.125014,-1.233690,0.874770,0.64978,-0.546810,-2.426220


# Models

In [871]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from xgboost import plot_importance

# XGBoost

In [865]:
train_xgb = xgb.DMatrix(X_train, label = y_train)
test_xgb = xgb.DMatrix(X_test, label = y_test)

In [866]:
param = {
    'max_depth': 100,
    'eta': 0.3,
    'objective': 'binary:logistic',
    'alpha':0.0001,
    'lambda': 0.01
}
epochs = 100
model_xgb = xgb.train(param, train_xgb, epochs)

In [ ]:
pred1 = model_xgb.predict(test_xgb)
pred_xgb = (pred > 0.50)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
print('Accuracy:')
print(accuracy_score(y_test,pred_xgb))
print()
print('Recall:')
print(recall_score(y_test,pred_xgb))
print()
print('Confusion:')
confusion_matrix(y_test,pred_xgb)

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 22
fig, ax = plt.subplots(figsize=(6,6))
xgb.plot_importance(model_xgb, max_num_features=5, height=0.5, ax=ax, color = 'orange')
plt.grid(b=None)
plt.title('Feature Importance')
plt.xlabel('F-Score')
plt.ylabel('Song Trait')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.show()

# SVM

In [491]:
model_svm = SVC(C=100000, gamma='auto')

In [771]:
model.fit(X_train, y_train)

In [ ]:
pred_svm = model_svm.predict(X_test)

In [492]:
print('Accuracy:')
print(accuracy_score(y_test,pred_svm))
print()
print('Recall:')
print(recall_score(y_test,pred_svm))
print()
print('Confusion:')
confusion_matrix(y_test,pred_svm)

SVC(C=100000, gamma='auto')